In [ ]:
# change direction to main folder to use path from config.json

import os
os.chdir('C:/Users/Domen/OneDrive/Projects/linkedIn-webscraper')
print(os.getcwd())


In [ ]:
# Importing necessary libraries
import winsound
import numpy as np
from bs4 import BeautifulSoup, NavigableString
import requests
from time import sleep
from random import randint
import math
from tqdm import tqdm
import pickle
import pandas as pd
from datetime import date
from datetime import datetime
import json
import zlib


from myModules import scraper as mybib # importing own library


%load_ext autoreload
%autoreload 2

In [ ]:
# load paths from config 

# print(os.getcwd())
with open("config/config.json", 'r') as f:
    config = json.load(f)
    
temp_data_path = config["temp_data_path"]
webscrap_data_path = config["webscrap_data_path"]   
sound_path = config["sound_path"]

# check paths
print(temp_data_path)
print(webscrap_data_path)
print(sound_path)

soup_file =  'soup_dict'
keyword_file = 'keyword_dict'
database_file = 'database'

### Import keywords

In [ ]:
keyword_list= ['data_analyst']
keyword_list

### Scrap Prep

In [ ]:
try:
    with open(f"{webscrap_data_path}{soup_file}.pkl", "rb") as file:
        soup_dict = pickle.load(file)
    with open(f"{temp_data_path}{soup_file}_backup.pkl", "wb") as file:
        pickle.dump(soup_dict, file)
except Exception as e:
    raise ValueError(f"Error occurred: {e} - soup_dict")      
        
# try:
#     with open(f"{webscrap_data_path}{keyword_file}.pkl", "rb") as file:
#         keyword_dict = pickle.load(file)
#     with open(f"{temp_data_path}{keyword_file}_backup.pkl", "wb") as file:
#         pickle.dump(soup_dict, file) 
# except Exception as e:
#     raise ValueError(f"Error occurred: {e} - keyword_dict")   

In [ ]:
print(len(soup_dict))
print(len(keyword_dict))


In [ ]:
print(soup_dict)
id_control = []
for item in soup_dict.keys():
    id_control.append(item)

In [ ]:
id_control

In [ ]:

try:
    for key in keyword_list:
        first_url = f"https://www.linkedin.com/jobs/search?keywords={key}&location=Stuttgart%2C%20Baden-W%C3%BCrttemberg%2C%20Germany&geoId=102473731&position=1&pageNum=0"
        key_name = key.replace('%20', " ")
            
        response = requests.get(first_url) # first request for keyword
        response.status_code # 200 status code means OK!
        soup = BeautifulSoup(response.content, "html.parser")
        
        number_of_results = soup.find('span', class_="results-context-header__job-count").text # check number of searching results
        numb = int(number_of_results.replace(",", "").replace("+", ""))

        backend_call_url_list = []
        backend_call_url_list = mybib.create_backend_links(first_url, numb, key_name) # create list with sublinks to select different pages 
        
        
        
        id_array = mybib.get_id_list(backend_call_url_list) # get job id's from all pages
        

        with tqdm(total=len(id_array), desc="Starting") as pbar:
        
            for id in id_array:
                counter = 0
                backupsave = [25, 50, 75, 100, 125, 150, 175, 200, 300] 
                if counter in backupsave:
                    new_dataframe = mybib.export_data(soup_dict, keyword_dict)
                    scraper_df = pd.DataFrame(columns=scraper_df.columns)
                    print("Updated dataframe")
                dynamic_text = f"id {id}" # text for tqdm progress bar status
                pbar.set_description(dynamic_text) # change text
                pbar.update(1)
                
                if str(id) not in id_control:
                    try:
                        
                        response = requests.get(f'https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{id}')
                        wait_time = randint(1,200)
                        sleep(wait_time/1000)
                        
                        dict2 = {}
                        dict2["scrap_date"] = date.today()
                        dict2["response"] = response
                        soup_dict[id] = dict2
                        try:
                            keyword_dict[id].append(key_name)
                        except:
                            keyword_dict[id] = [key_name]


                    except Exception as e:
                        raise ValueError(f"Error occurred: {e} getting data from id {id}")
                        
                else:
                    pbar.set_description(f"Will skip {id} because is already in the dataset.")
                    try:
                        keyword_dict[id].append(key_name)
                    except:
                        keyword_dict[id] = [key_name]
                counter += 1    
        new_dataframe = mybib.export_data(soup_dict, keyword_dict)
        print("\n \n")
except Exception as e:
    winsound.PlaySound(f"{sound_path}alarm3.wav", winsound.SND_FILENAME)
    raise ValueError(f"Error occurred: {e}")
    
winsound.PlaySound(f"{sound_path}alarm3.wav", winsound.SND_FILENAME)

In [ ]:
display(len(soup_dict))
display(len(keyword_dict))

In [ ]:
new_dataframe['id'].value_counts()


In [ ]:
df_unique = new_dataframe.drop_duplicates(subset=['id'])
df_unique


In [ ]:

with open(f"webscraper/webscrap_data/{database_file}.pkl" , "wb") as f:
    pickle.dump(df_unique, f)

In [ ]:
columns = new_dataframe.columns

for col in columns:
    null = new_dataframe[col].isna().sum()
    print(f"{round(null / len(new_dataframe[col]) * 100,1)}%    {col}:")

In [ ]:
keywords_input = ['Data%20Analyst']
keywords = []



In [ ]:
first_url = f"https://www.linkedin.com/jobs/search?keywords=%7Bkey%7D&location=Berlin%2C%20Berlin%2C%20Germany&locationId=&geoId=106967730&f_TPR=&f_PP=106967730&distance=25&f_JT=F&f_E=2%2C3%2C4&position=1&pageNum=0"



In [ ]:

response = requests.get(f'https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/3717202675')


In [ ]:
soup_dict['3719620327']

In [ ]:
response = soup_dict['3719620327']

soup = BeautifulSoup(response.content, "html.parser")
soup

In [ ]:
soup.find("section",{"class":"show-more-less-html"}).text.strip()

In [ ]:
soup_dict

In [ ]:
soup_dict.keys()

In [ ]:
new_soup_dict = {}

In [ ]:
for key in soup_dict:
    temp = {}
    
    dict2 = {}
    dict2["scrap_date"] = date.today()
    dict2["response"] = soup_dict[key]
    new_soup_dict[key] = dict2

In [ ]:
new_soup_dict

In [ ]:
with open("webscraper/webscrap_data/soup_dict.pkl", "wb") as f:
    pickle.dump(new_soup_dict, f)

In [ ]:
id_list = mybib.get_id_list(backend_call_url_list)

In [ ]:
len(id_list)